In [9]:
import csv
import pandas as pd
from sqlalchemy import create_engine, text

In [5]:
from mysql.connector import MySQLConnection, Error

In [6]:
#Making connection to SQL server
def make_connection():
    try:
        conn = MySQLConnection(host='IES-ADS-ClassDB.sjsu.edu',
                               database='querycrew_db',
                               user='querycrew_user',
                               password='Pomegranate_746')
        
        if conn.is_connected():
            print('Connected to the MySQL database!')
            
            return conn
                
    except Error as e:
        print('Connection failed.')
        print(e)
        
        return None

In [7]:
conn = make_connection()
cursor = conn.cursor()

Connection failed.
2003: Can't connect to MySQL server on 'IES-ADS-ClassDB.sjsu.edu:3306' (Errno 8: nodename nor servname provided, or not known)


AttributeError: 'NoneType' object has no attribute 'cursor'

In [ ]:
cursor.execute('DROP TABLE IF EXISTS location')

In [ ]:
cursor.execute('DROP TABLE IF EXISTS vehicle')

In [ ]:
#This is for the one-to-one tables. The primary key is VIN(1-10) 
#Creating a vehicle table

sql = ( """
        CREATE TABLE vehicle
        (
            vin varchar(32),
            make varchar(32),
            model varchar(32),
            model_year int,
            PRIMARY KEY(vin)
        )
        """
      )
cursor.execute(sql)

In [ ]:
#FIGURE OUT A PRIMARY KEY FOR THIS TABLE
sql = ( """
        CREATE TABLE location
        (
            dol_id int,
            vin varchar(20),
            county varchar(32),
            city varchar(32),
            state varchar(32),
            zip_code int,
            PRIMARY KEY(dol_id),
            FOREIGN KEY (vin) REFERENCES vehicle(vin)
        )
        """
      )

cursor.execute(sql);

In [ ]:
#For both of these two tables created, the primary key is the vin, and the foriegn key is in the location table and it is the vin as well
sql_insert_vehicle = (  
    """
    INSERT INTO vehicle (vin, model_year, make, model)
    VALUES (%s, %s, %s, %s)
    """
)

In [ ]:
sql_insert_location = ( 
    """
    INSERT INTO location (dol_id, vin, county, city, state, zip_code)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
)

In [ ]:
def transform_vehicle(row):
    #Converting the model_year column and zip-code column to an int
    try:
      row[1] = int(row[1])
    except ValueError:
      row[4] = None
      row[1] = int(row[1])

    return row

In [ ]:
def transform_location(row):
    try:
      row[0] = int(row[0])
      row[4] = int(row[4])
    except ValueError:
      row[0] = None
      row[4] = None

    return row

In [ ]:
vehicle_cols = [0, 5, 6, 7]
location_cols = [13, 0, 1, 2, 3, 4]
first = True
seen_vins = set()  

# Read CSV data into a list before processing
with open('Electric_Vehicle_Population_Data.csv', newline='') as csv_file:
    data = list(csv.reader(csv_file, delimiter=',', quotechar='"'))  

for row in data:
    if not first:
        vin = row[vehicle_cols[0]]  

        # Skips over the duplicates found in the file
        if vin in seen_vins:
            continue  
        seen_vins.add(vin)  

        # Here I am checking if the VIN already exists within the database for vehicle
        cursor.execute("SELECT COUNT(*) FROM vehicle WHERE vin = %s", (vin,))
        if cursor.fetchone()[0] > 0:
            continue 
        #This is where we are identifying the specific columns 
        vehicle_row = [row[i] for i in vehicle_cols]
        location_row = [row[i] for i in location_cols]
          
        #Inserting into repsective tables
        cursor.execute(sql_insert_vehicle, vehicle_row)
        cursor.execute(sql_insert_location, location_row)

    first = False  

conn.commit()


In [ ]:
sql = (  
    """
    SELECT * FROM vehicle LIMIT 25
    """
)
cursor.execute(sql)
results = cursor.fetchall()



for row in results:
    print(row)

conn.commit()

In [ ]:
sql = (  
    """
    SELECT * FROM location LIMIT 25
    """
)
cursor.execute(sql)
results = cursor.fetchall()

if results:
    for row in results:
        print(row)
conn.commit()

In [ ]:
df = data
# Extracting 50 rows of "Make" and "Model Year" from CSV (using .loc for label indexing the CSV file)
#We only want the UNIQUE values of Make to present the One-to-Many Relationship with the second table.
make_and_model_year_data = df.loc[:49, ["Make", "Model Year"]].groupby("Make").first().reset_index()

# Extracting 50 rows of "Make", "Model", and "Vehicle Type" from CSV file.
make_model_type_data = df.loc[:49, ["Make", "Model", "Electric Vehicle Type"]]

print("Make and Model Year Data")
print(make_and_model_year_data.head(25))
 
print("\n Make Model Type Data")
print(make_model_type_data.head(25))

In [ ]:
#Drop table if it already exists
cursor.execute('DROP TABLE IF EXISTS make_model_year')
cursor.execute('DROP TABLE IF EXISTS make_model_type')

#Creating the tables to hold make, year, model, and type of vehicles
make_model_year = ( """
        CREATE TABLE make_model_year
        (
            make       varchar(32),
            year      int,
            PRIMARY KEY (make)
        )
        """
      )
make_model_type = ( """
        CREATE TABLE make_model_type
        (
            make       varchar(32),
            model   varchar(32),
            type        varchar(64)
        )
        """
      )

cursor.execute(make_model_year);
cursor.execute(make_model_type);

print("Tables created successfully!")

In [ ]:
#Inserting values into the respective tables
sql_insert_make_model_year = (  
    """
    INSERT INTO make_model_year (make, year)
    VALUES (%s, %s)
    """
)

sql_insert_make_model_type = (  
    """
    INSERT INTO make_model_type (make, model, type)
    VALUES (%s, %s, %s)
    """
)

#Looping and iterating through the respective DataFrames to insert values into created Databases.
for row in make_and_model_year_data.itertuples(index=False, name=None):
    cursor.execute(sql_insert_make_model_year, row)
print("Inserted make and model_year data successfully!")
for row in make_model_type_data.itertuples(index=False, name=None):
    cursor.execute(sql_insert_make_model_type, row)
print("Inserted make, model, and type successfully!")


In [ ]:
#Displaying the one-to-many relationship
#where one MAKE can have MULTIPLE MODELS

print("Make-Model-Year Table")
sql_make_model_year = (
    """
    SELECT * FROM make_model_year LIMIT 25;
    """
)
cursor.execute(sql_make_model_year)
result_data = cursor.fetchall()
for r in result_data:
    print(r)

print("\nMake-Model-Type Table")
sql_make_model_type = (
    """
    SELECT * FROM make_model_type LIMIT 25;
    """
)
cursor.execute(sql_make_model_type)
result_data_2 = cursor.fetchall()
for r2 in result_data_2:
    print(r2)


In [ ]:
# Select and clean Makes data (limit to 100 rows)
make_data = df[["Make"]].drop_duplicates().head(100)
make_data.columns = ["make_name"]

# Select and clean Counties data (limit to 100 rows)
county_data = df[["County"]].drop_duplicates().head(100)
county_data.columns = ["county_name"]

# Extract Make-County Relationships (limit to 100 rows)
make_county_data = df[["Make", "County"]].drop_duplicates().head(100)
make_county_data.columns = ["make_name", "county_name"]

# Display data
print("Makes Data (First 25 Rows):")
print(make_data.head(25))

print("\nCounties Data (First 25 Rows):")
print(county_data.head(25))

print("\nMake-County Relationships (First 25 Rows):")
print(make_county_data.head(25))

In [ ]:
with engine.connect() as conn:
    # Drop existing tables
    conn.execute(text("DROP TABLE IF EXISTS make_counties;"))
    conn.execute(text("DROP TABLE IF EXISTS makes;"))
    conn.execute(text("DROP TABLE IF EXISTS counties;"))

    # Create Makes Table
    
    conn.execute(text("""
        CREATE TABLE makes (
            make_id INT AUTO_INCREMENT PRIMARY KEY,  -- makes_id is the Primary Key for makes table
            make_name VARCHAR(50) UNIQUE NOT NULL    -- Make names must be unique
        )
    """))

    # Create Counties Table
    conn.execute(text("""
        CREATE TABLE counties (
            county_id INT AUTO_INCREMENT PRIMARY KEY,  -- county_id is the Primary Key for counties table
            county_name VARCHAR(100) UNIQUE NOT NULL   -- Each county has a unique name
        )
    """))

    # Create Make-County Linking Table
    conn.execute(text("""
        CREATE TABLE make_counties (
            make_id INT,    -- Foreign Key referencing the makes table
            county_id INT,  -- Foreign Key referencing the counties table
            PRIMARY KEY (make_id, county_id),  -- Composite Primary Key that ensures unique pairs
            FOREIGN KEY (make_id) REFERENCES makes(make_id) ON DELETE CASCADE,  -- Deletes linked records if make is removed
            FOREIGN KEY (county_id) REFERENCES counties(county_id) ON DELETE CASCADE  -- Ensures valid county refrences
        )
    """))

    print("Tables created successfully!")


In [ ]:
# Insert Makes Data
with engine.connect() as conn:
    for _, row in make_data.iterrows():
        conn.execute(text("""
            INSERT IGNORE INTO makes (make_name) 
            VALUES (:make_name)
        """), row.to_dict())

    print("Makes inserted successfully!")

# Insert Counties Data
with engine.connect() as conn:
    for _, row in county_data.iterrows():
        conn.execute(text("""
            INSERT IGNORE INTO counties (county_name) 
            VALUES (:county_name)
        """), row.to_dict())

    print("Counties inserted successfully!")

# Insert Make-County Relationships
with engine.connect() as conn:
    for _, row in make_county_data.iterrows():
        conn.execute(text("""
            INSERT IGNORE INTO make_counties (make_id, county_id)
            SELECT m.make_id, c.county_id
            FROM makes m, counties c
            WHERE m.make_name = :make_name AND c.county_name = :county_name
        """), row.to_dict())

    print("Make-County relationships inserted successfully!")


In [ ]:
# Query and display first 25 rows from each table
with engine.connect() as conn:
    print("\nMakes Table (First 25 Rows):")
    result = conn.execute(text("SELECT * FROM makes LIMIT 25;"))
    for row in result:
        print(row)

    print("\nCounties Table (First 25 Rows):")
    result = conn.execute(text("SELECT * FROM counties LIMIT 25;"))
    for row in result:
        print(row)

    print("\nMake-County Relationships (First 25 Rows):")
    result = conn.execute(text("""
        SELECT m.make_name, c.county_name 
        FROM make_counties mc
        JOIN makes m ON mc.make_id = m.make_id
        JOIN counties c ON mc.county_id = c.county_id
        LIMIT 25;
    """))
    for row in result:
        print(row)


In [ ]:
#Here are the 5 sql queries joining the above tables:

queries = [
    "SELECT v.vin, v.make, v.model, v.model_year, m.year FROM vehicle v JOIN make_model_year m ON v.make = m.make;",
    "SELECT v.vin, v.make, v.model, mmt.type FROM vehicle v JOIN make_model_type mmt ON v.make = mmt.make AND v.model = mmt.model;",
    "SELECT mk.make_name, c.county_name FROM makes mk JOIN make_counties mc ON mk.make_id = mc.make_id JOIN counties c ON mc.county_id = c.county_id;",
    "SELECT v.make, COUNT(v.vin) AS total_vehicles FROM vehicle v GROUP BY v.make;",
    "SELECT v.vin, v.make, v.model, v.model_year, c.county_name FROM vehicle v JOIN makes mk ON v.make = mk.make_name JOIN make_counties mc ON mk.make_id = mc.make_id JOIN counties c ON mc.county_id = c.county_id WHERE c.county_name = 'Los Angeles County';"
]

with engine.connect() as connection:
    for query in queries:
        result = connection.execute(text(query))
        print(result.fetchall())

In [ ]:
cursor.close()
conn.close()